In [1]:
import numpy as np
import pandas as pd
from pyBKT.models import Model
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
model = Model(seed = 60, num_fits = 1)

In [3]:
model.fetch_dataset("https://raw.githubusercontent.com/lishaparmar13/H5P-Enhanced-BKT/main/Above16-CS.csv",'.')

In [4]:
df2= pd.read_csv(r"Above16-CS.csv", encoding ='latin')
df2.head(5)

,ï»¿Row,Anon Student Id,Age,Problem Hierarchy,question_id,Step Start Time,Step End Time,Correct First Attempt,Step Duration (sec),Answer Type,KC(Default),skill_id,Opportunity (Default),template_id
0,6,6,16+,1,3,15-07-2023 15:03,15-07-2023 15:04,0,60,Drag & Drop,Algebra,1,1,1
1,7,7,16+,1,3,15-07-2023 15:39,15-07-2023 15:40,0,6,Drag & Drop,Algebra,1,1,1
2,9,9,16+,1,3,15-07-2023 17:34,15-07-2023 17:35,0,45,Drag & Drop,Algebra,1,1,1
3,10,10,16+,1,3,15-07-2023 20:00,15-07-2023 20:01,0,49,Drag & Drop,Algebra,1,1,1
4,14,14,16+,1,3,17-07-2023 22:19,17-07-2023 22:20,0,41,Drag & Drop,Algebra,1,1,1


In [5]:
model.fit(data = df2)

In [6]:
train_df, test_df = train_test_split(df2, test_size=0.2, random_state=42)

In [7]:
model.fit(data = train_df)
print("Fitted Skills:\n%s" % '\n'.join(model.coef_.keys()))

Fitted Skills:
Numbers and Geometry and Fractions
Numbers and Algebra and Fractions
Algebra and Fractions
Algebra
Numbers and Percentages and Ratio and Proportion
Fractions
Numbers and Geometry and Percentages
Geometry
Ratio and Proportion


In [8]:
# Evaluate with the default RMSE then specify AUC.
model.fit(data= df2)
training_rmse = model.evaluate(data = df2)
training_auc = model.evaluate(data = df2, metric = 'auc')
print("Training RMSE: %f" % training_rmse)
print("Training AUC: %f" % training_auc)

Exception ignored in: <Finalize object, dead>
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\multiprocessing\util.py", line 224, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\multiprocessing\pool.py", line 713, in _terminate_pool
    p.terminate()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\multiprocessing\process.py", line 133, in terminate
    self._popen.terminate()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\multiprocessing\popen_spawn_win32.py", line 123, in terminate
    _winapi.TerminateProcess(int(self._handle), TERMINATE)
PermissionError: [WinError 5] Access is denied


Training RMSE: 0.483622
Training AUC: 0.611049


In [9]:
#define own metric
def mae(true_vals, pred_vals):
  """ Calculates the mean absolute error. """
  return np.mean(np.abs(true_vals - pred_vals))

training_mae = model.evaluate(data = df2, metric = mae)
print("Training MAE: %f" % training_mae)

Training MAE: 0.467985


In [10]:
model = Model(seed = 60, num_fits = 1)
model.fit(data = train_df)
preds = model.predict(data = test_df)
preds[['Anon Student Id', 'KC(Default)', 'Correct First Attempt', 
       'correct_predictions', 'state_predictions']].head(10)

,Anon Student Id,KC(Default),Correct First Attempt,correct_predictions,state_predictions
0,6,Algebra,0,0.24991,0.15696
30,6,Ratio and Proportion,1,0.50000,0.24505
19,7,Numbers and Geometry and Fractions,0,0.50000,0.78102
13,7,Numbers and Geometry and Percentages,1,0.50000,0.13940
39,10,Algebra and Fractions,1,0.50000,0.90078
21,10,Numbers and Geometry and Fractions,1,0.50000,0.78102
34,14,Ratio and Proportion,1,0.50000,0.24505
52,14,Numbers and Algebra and Fractions,1,0.50000,0.57147
40,14,Algebra and Fractions,0,0.50000,0.90078
29,25,Algebra,1,0.24991,0.15696


In [11]:
# Let's sanity check that we have only trained on the skills that we 
# specified in the call to fit! Note that while it is possible for a 
# BKT prediction to be 0.5 exactly, it is unlikely.
preds[preds['correct_predictions'] != 0.5]['KC(Default)'].unique()

array(['Algebra'], dtype=object)

In [12]:
model.evaluate(data = df2)

0.4958048758608929

In [13]:
model.evaluate(data=df2, metric=['rmse','accuracy','auc'])

[0.4958048758608929, 0.6, 0.5809151785714286]

In [14]:
# Note that folds is an optional parameter as well as the model 
# variant, seed, and crossvalidated metric.
# By default, we crossvalidate on all skills separately.
model.crossvalidate(data = df2, folds = 5)

,rmse
skill,
Algebra,0.44849
Fractions,0.50000
Numbers and Geometry and Percentages,0.50000
Numbers and Geometry and Fractions,0.50000
Ratio and Proportion,0.50000
Algebra and Fractions,0.50000
Geometry,0.50000
Numbers and Algebra and Fractions,0.50000
Numbers and Percentages and Ratio and Proportion,0.50000


In [15]:
model = Model(seed = 60, num_fits = 1)
# Try this with a different skill or metric by replacing the lines below.
skill = 'Algebra'
metric = 'rmse'

simple_cv = model.crossvalidate(data = df2, skills = skill, 
                                metric = metric)
simple_cv

,rmse
skill,
Algebra,0.44862


In [16]:
model = Model(seed = 60, num_fits = 1)
skill = 'Algebra'

multilearn_cv = model.crossvalidate(data = df2, skills = skill,
                                    multilearn ="Answer Type", forgets = True, 
                                    metric = metric)
multilearn_cv

,rmse
skill,
Algebra,0.44861


In [17]:
model = Model(seed = 60, num_fits = 1)
multilearn_cv = model.crossvalidate(data= df2, skills = skill,
                                    multilearn ="question_id", 
                                    metric = metric)
multilearn_cv

,rmse
skill,
Algebra,0.44882


In [18]:
model = Model(seed = 60, num_fits = 1)
# The multiprior model generates different priors based on the first 
# response of each student.
multiprior_cv = model.crossvalidate(data= df2, skills = skill,
                                    multiprior = True, metric = metric,
                                    folds = 3)

model = Model(seed = 60, num_fits = 1)
multipair_cv = model.crossvalidate(data = df2, skills = skill,
                                   multipair = True, metric = metric,
                                   folds = 3)

pd.concat([multiprior_cv, multipair_cv], axis = 0)

,rmse
skill,
Algebra,0.48162
Algebra,0.60578


In [19]:
model = Model(seed = 60, num_fits = 1)
# We combine the fifth parameter, forgets, with the previous multilearn
# and multiguess/slip models for a combo model.
combo_cv = model.crossvalidate(data = df2, skills = skill,
                               forgets = True, multilearn = True, 
                               multigs = True, metric = metric)
combo_cv

,rmse
skill,
Algebra,0.46116


In [20]:
skill = 'Geometry'

model.coef_ = {skill: {'prior': 1e-40}}
model.coef_

{'Geometry': {'prior': 1e-40}}

In [21]:
# Train the model with the pre-initialized parameters.
model.fit(data = df2, multigs = True)
low_prior_auc = model.evaluate(data = df2, metric = 'auc')

# We can obtain the prior value by indexing into the model.coef_
# dictionary with the skill and parameter names. 
print("Fitted Prior Value: %f" % model.coef_[skill]['prior'])
print("Training AUC: %f" % low_prior_auc)

Fitted Prior Value: 0.000000
Training AUC: 0.611049


In [22]:
# Initialize the prior to be more reasonable.
model.coef_ = {skill: {'prior': 0.5}}
model.fit(data= df2, multigs = True)
normal_prior_auc = model.evaluate(data= df2, metric = 'auc')

# Print the fitted prior value and RMSE.
print("Fitted Prior Value: %f" % model.coef_[skill]['prior'])
print("Training AUC: %f" % normal_prior_auc)

Fitted Prior Value: 0.324920
Training AUC: 0.611049


In [23]:
model.fit(data=df2, forgets = True, multilearn = True, skills=skill, 
          multigs = True)
model.params()

value
skill    param   class          
Geometry prior   default 0.26468
         learns  6       1.00000
         guesses 6       0.50000
         slips   6       0.50000
         forgets 6       0.00000

In [24]:
model.fit(data=df2, forgets = True, multilearn = True, 
          multigs = True)
model.params()

value
skill                                            param   class          
Algebra                                          prior   default 0.17118
                                                 learns  1       0.11823
                                                         3       1.00000
                                                 guesses 1       0.00040
                                                         3       0.31602
                                                 slips   1       0.02830
                                                         3       0.00029
                                                 forgets 1       0.00043
                                                         3       0.00000
Fractions                                        prior   default 0.57225
                                                 learns  1       1.00000
                                                 guesses 1       0.50000
                                                 slips   1       0.50000
                                                 forgets 1       0.00000
Numbers and Geometry and Percentages             prior   default 0.69796
                                                 learns  1       1.00000
                                                 guesses 1       0.50000
                                                 slips   1       0.50000
                                                 forgets 1       0.00000
Numbers and Geometry and Fractions               prior   default 0.36301
                                                 learns  2       1.00000
                                                 guesses 2       0.50000
                                                 slips   2       0.50000
                                                 forgets 2       0.00000
Ratio and Proportion                             prior   default 0.46827
                                                 learns  4       1.00000
                                                 guesses 4       0.50000
                                                 slips   4       0.50000
                                                 forgets 4       0.00000
Algebra and Fractions                            prior   default 0.14129
                                                 learns  5       1.00000
                                                 guesses 5       0.50000
                                                 slips   5       0.50000
                                                 forgets 5       0.00000
Geometry                                         prior   default 0.23911
                                                 learns  6       1.00000
                                                 guesses 6       0.50000
                                                 slips   6       0.50000
                                                 forgets 6       0.00000
Numbers and Algebra and Fractions                prior   default 0.71911
                                                 learns  6       1.00000
                                                 guesses 6       0.50000
                                                 slips   6       0.50000
                                                 forgets 6       0.00000
Numbers and Percentages and Ratio and Proportion prior   default 0.78097
                                                 learns  6       1.00000
                                                 guesses 6       0.50000
                                                 slips   6       0.50000
                                                 forgets 6       0.00000